# Task 3: Credit Risk Analysis

Prototype PD and Expected Loss model for retail loan portfolios.

## Load Data

In [ ]:

import pandas as pd
import numpy as np

df = pd.read_csv("/mnt/data/Task 3 and 4_Loan_Data.csv")
df.head()


## Prepare Features and Target

In [ ]:

y = df['default']
X = df.drop(columns=['default'])

loan_amount = X['loan_amount']
X = X.drop(columns=['loan_amount'])


## Train PD Model (Logistic Regression)

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression())
])

model.fit(X_train, y_train)


## Model Evaluation

In [ ]:

from sklearn.metrics import roc_auc_score

pd_preds = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, pd_preds)

print(f"ROC-AUC Score: {auc:.3f}")


## Expected Loss Function

In [ ]:

def expected_loss(
    borrower_features,
    loan_amount,
    model,
    recovery_rate=0.10
):
    pd_value = model.predict_proba(borrower_features)[0, 1]
    lgd = 1 - recovery_rate
    el = pd_value * loan_amount * lgd
    return el


## Example Calculation

In [ ]:

sample_borrower = X.iloc[[0]]
sample_loan_amount = loan_amount.iloc[0]

el = expected_loss(
    borrower_features=sample_borrower,
    loan_amount=sample_loan_amount,
    model=model
)

print(f"Expected Loss: ${el:,.2f}")
